In [1]:
# Import the research library 
import logging
import benchmarklib
from benchmarklib import BenchmarkDatabase, CompileType
from benchmarklib.algorithms import GroverRunner, GroverConfig, calculate_grover_iterations
from benchmarklib.compilers import TruthTableCompiler, XAGCompiler, QCFCompiler
from benchmarklib.pipeline import PipelineCompiler
from benchmarklib.pipeline.synthesis import QuantumMPC
from qiskit_ibm_transpiler import generate_ai_pass_manager

from clique import CliqueProblem, CliqueTrial, CliqueOracleTrial, run_clique_benchmark, logger as clique_logger

benchmarklib.setup_logging(logging.INFO)

clique_logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
handler.setLevel(logging.INFO)
clique_logger.addHandler(handler)
clique_logger.propagate = False


from compiler_configs import compiler_qmpc_ai_test, compiler_qmpc_standard, compiler_qcf, compiler_xag

db = BenchmarkDatabase("clique.db", CliqueProblem, CliqueTrial)

benchmarklib.core.database - INFO - Database initialized: clique.db (CLIQUE)


In [ ]:
from benchmarklib.pipeline.config import PipelineConfig
from sqlalchemy import select, delete
#with db.session() as session:
#    stmt = delete(PipelineConfig).where(PipelineConfig.id >= 0)
#    session.execute(stmt)
#    session.commit()
db.query(select(PipelineConfig))

In [2]:
# Load Qiskit
from qiskit_ibm_runtime import QiskitRuntimeService, Batch
from dotenv import load_dotenv, find_dotenv
import os

service = QiskitRuntimeService()
backend = service.backend(name="ibm_rensselaer")

In [3]:
nodes_range = range(3, 9)
edge_prob_range = range(1, 99, 5)

In [ ]:
#QCFCompiler(), XAGCompiler(), compiler_qmpc_ai_test
for compiler in [compiler_qmpc_standard,]:
    run_clique_benchmark(
        db=db,
        compiler=compiler,
        backend=backend,
        nodes_iter=nodes_range,
        num_problems=50,
        shots=10**3,
        max_problems_per_job=5,
        save_circuits=True,
    )

In [ ]:
oracle_db = BenchmarkDatabase("clique.db", CliqueProblem, CliqueOracleTrial)
for compiler in [compiler_xag, compiler_qcf]:
    run_clique_benchmark(
        db=oracle_db,
        compiler=compiler,
        backend=backend,
        nodes_iter=nodes_range,
        num_problems=20,
        shots=10**3,
        max_problems_per_job=5,
        save_circuits=True,
    )

benchmarklib.core.database - INFO - Database initialized: clique.db (CLIQUE)
Compiling problem ID 1 with 3 nodes.
Compiling problem ID 7 with 3 nodes.
Compiling problem ID 4 with 3 nodes.
Skipping (nodes=4) -- already have 30 instances
benchmarklib.runners.queue - WARNING - Auto-submitting 15 remaining circuits
benchmarklib.runners.queue - INFO - Attempting to submit job with 15 circuits
benchmarklib.runners.queue - INFO - finished batch (1 jobs submitted)
